# Bases de connaissance : interroger Wikidata à l'aide de requêtes SPARQL

## Imports

In [2]:
from datetime import datetime as dt
from SPARQLWrapper import SPARQLWrapper, JSON

## Obtenir la liste des journalistes belges qui étaient également écrivain

In [26]:
# Retrieve results from SPARQL
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql = SPARQLWrapper(endpoint)

# P27 = country of citizenship 
# Q31 = Belgium
# P106 = Opccupation
# Q82955 = Politician
# P569 = date of birth
# P570 = date of death

# Q1930187 = journaliste
# Q36180 = écrivain

statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
    ?person wdt:P27 wd:Q31 .
    ?person wdt:P106 wd:Q1930187 .
    ?person wdt:P106 wd:Q36180 .
    ?person wdt:P569 ?dateBirth .
    OPTIONAL {?person wdt:P570 ?dateDeath .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?personLabel
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} Belgian journalists found\n")
print(rows[:10]) 


279 Belgian journalists found

[{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q179296'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Achille Delattre'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1879-08-24T00:00:00Z'}, 'dateDeath': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1964-07-13T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3605493'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Adolphe Bartels'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1802-08-24T00:00:00Z'}, 'dateDeath': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1862-01-01T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q14870041'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'A

## Filtrer pour n'afficher que les noms des journalistes écrivains belges mort lors de la seconde guerre mondiale

In [38]:
date_filter_bottom = 1940
date_filter_top = 1945
count = 0

filtered_rows = [row for row in rows]
date_format = "%Y-%m-%dT%H:%M:%SZ"

for row in filtered_rows[:]:
    try:
        birth_date = dt.strptime(row['dateBirth']['value'], date_format)
        birth_year = birth_date.year
    except ValueError:
        birth_year = 0
    try:
        death_date = dt.strptime(row['dateDeath']['value'], date_format)
        death_year = death_date.year
    except ValueError: # unknown death date
        death_year = 0
    except KeyError: # still alive
        death_year = 9999
        #J'ai du changer les valeurs des excepts pou pouvoir faire des comparaisons de int entre int. Car avant on mettait des str dans les excepts et ça faisait planter la comparaison.
    if date_filter_bottom < death_year and death_year < date_filter_top: 
        #j'ai dû descendre le if du filtered_rows ici car sinon je n'avais pas accès aux valeurs death_year et birht_year qui sont dans la boucle for
        print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")
        count += 1

print(f"\nIl y a {count} résultats")

Gustave Fuss-Amoré (1877-1944)
Horace Van Offel (1876-1944)
Jean Gustave Schoup (1893-1944)
Jean Gustave Schoup (1893-1944)
Joris d'Hanswyck (1878-1942)
Lode Zielens (1901-1944)
Maurice Des Ombiaux (1868-1943)
Paul Colin (1890-1943)
Paul Colin (1890-1943)
Paul Colin (1890-1943)
Édouard Baudry (1905-1943)

Il y a 11 résultats


## Pour en savoir plus

- [Le projet Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page)
- [Aide à la construction de requêtes](https://query.wikidata.org/)